# Holder Wallet Conformance Flows (v3) - CTWalletSamePreAuthorised

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from eudi_wallet.did_key import KeyDid, PublicKeyJWK
import uuid

# generate crypto seed
crypto_seed = b'ebsitests'

key_did = KeyDid(seed=crypto_seed)

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9Kbowo1MYpCJwNfzfFggXuWxdPAdAWhkp9XskU2cir7r21AxqN2TMvMTUS1FmaNWLm2esa8FLwZ35i5mRzUygDQdZRS1gC42Am9Doyh68Hp1Ksap9e9xzebDYFhhLgXTksgnG


## 1.1 Initiate Credential Issuance

In [3]:
from eudi_wallet.util import parse_query_string_parameters_from_url
from eudi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletSamePreAuthorised
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
# qr_code_data = "openid-credential-offer:?credential_offer_uri=https://wallet2.ngrok.io/issuer/credential-offer/3f7b72ce-2bf4-43e7-958c-a379761521e7"
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[23:35:49] Credential offer URI:                                                                   ]8;id=5898;file:///tmp/ipykernel_70125/2236403151.py\2236403151.py]8;;\:]8;id=518470;file:///tmp/ipykernel_70125/2236403151.py#16\16]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/2f22e702-497d-41eb-81                 
           e8-e14ef8f5b3ad                                                                                         

{'credential_issuer': 'https://api-conformance.ebsi.eu/conformance/v3/issuer-mock', 'credentials': [{'format': 'jwt_vc', 'types': ['VerifiableCredential', 'VerifiableAttestation', 'CTWalletSamePreAuthorised'], 'trust_framework': {'name': 'ebsi', 'type': 'Accreditation', 'uri': 'TIR link towards accreditation'}}], 'grants': {'urn:ietf:params:oauth:grant-type:pre-authorized_code': {'pre-authorized_code': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlRNdk1UVVMxRm1hTldMbTJlc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emViRFlGaGhMZ1hUa3NnbkciLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaW

[23:35:50] Credential offer:                                                                       ]8;id=94308;file:///tmp/ipykernel_70125/2236403151.py\2236403151.py]8;;\:]8;id=176686;file:///tmp/ipykernel_70125/2236403151.py#19\19]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletSamePreAuthorised'                                                             
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'urn:ietf:params:oauth:grant-type:pre-authorized_code': {                                       
                       'pre-authorized_code':                                                                      
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBe                 
           HFOMlRNdk1UVVMxRm1hTldMbTJlc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5                 
           ZTl4emViRFlGaGhMZ1hUa3NnbkciLCJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3J                 
           lZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS                 
           5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50a                 
           WFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lUHJlQXV0aG9yaXNlZCJdfV0sImlhdCI6                 
           MTY5MDk5OTU0OSwiZXhwIjoxNjkwOTk5ODQ5LCJpc3MiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV                 
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiYXVkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS                 
           5ldS9jb25mb3JtYW5jZS92My9hdXRoLW1vY2siLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1d                 
           U1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNp                 
           cjdyMjFBeHFOMlRNdk1UVVMxRm1hTld

[23:35:51] Credential issuer configuration:                                                        ]8;id=509627;file:///tmp/ipykernel_70125/2236403151.py\2236403151.py]8;;\:]8;id=344364;file:///tmp/ipykernel_70125/2236403151.py#22\22]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

           Authorization server configuration:                                                     ]8;id=132948;file:///tmp/ipykernel_70125/2236403151.py\2236403151.py]8;;\:]8;id=413531;file:///tmp/ipykernel_70125/2236403151.py#25\25]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Exchange code for access token

In [5]:
from eudi_wallet.siop_auth.util import exchange_pre_authorized_code_for_access_token

token_uri = auth_server_configuration.token_endpoint
user_pin = "4695"
access_token_response = await exchange_pre_authorized_code_for_access_token(token_uri, user_pin, credential_offer.grants.get("urn:ietf:params:oauth:grant-type:pre-authorized_code").get("pre-authorized_code"))
console.log("Access token response: ", access_token_response)

{'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjZlYjE0YzcyLWNmYTktNDBiOC04OGIwLWU3ZGRlZjliZjgxOSIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lUHJlQXV0aG9yaXNlZCJdfV0sImNfbm9uY2UiOiIwNGU1ZTZkMC0zY2Q5LTRhYmUtYTE5Yy0zNjEwNjI1NjVkYzEiLCJjX25vbmNlX2V4cGlyZXNfaW4iOjg2NDAwLCJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajlLYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlRNdk1UVVMxRm1hTldMbTJlc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emViRFlGaGhMZ1hUa3NnbkcifSwiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3JtYW5jZS92My9hdXRoLW1vY2siLCJhdWQiOlsiaHR0cHM6Ly9hcGktY29

[23:36:37] Access token response:                                                                   ]8;id=949423;file:///tmp/ipykernel_70125/3067923722.py\3067923722.py]8;;\:]8;id=398851;file:///tmp/ipykernel_70125/3067923722.py#6\6]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYW                
           VZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjZlYjE0YzcyLWNmYTktNDBiOC04OGIwLWU3ZGR                
           lZjliZjgxOSIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGV                
           udGlhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnN                
           pLmV1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lUHJlQXV0aG9yaXNlZCJdfV0sImNfbm9uY2UiOiI                
           wNGU1ZTZkMC0zY2Q5LTRhYmUtYTE5Yy0zNjEwNjI1NjVkYzEiLCJjX25vbmNlX2V4cGlyZXNfaW4iOjg2NDAwLCJ                
           jbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajl                
           LYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlRNdk1UVVMxRm1hTldMbTJ                
           lc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emViRFlGaGhMZ1hUa3Nnbkc                
           ifSwiaXNzIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3JtYW5jZS92My9hdXRoLW1vY2s                
           iLCJhdWQiOlsiaHR0cHM6Ly9hcGktY29uZm9ybWFuY2UuZWJzaS5ldS9jb25mb3JtYW5jZS92My9pc3N1ZXItbW9                
           jayJdLCJzdWIiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1BnWW95dHlrVVozZXlxaHQxajl                
           LYm93bzFNWXBDSndOZnpmRmdnWHVXeGRQQWRBV2hrcDlYc2tVMmNpcjdyMjFBeHFOMlRNdk1UVVMxRm1hTldMbTJ                
           lc2E4Rkx3WjM1aTVtUnpVeWdEUWRaUlMxZ0M0MkFtOURveWg2OEhwMUtzYXA5ZTl4emViRFlGaGhMZ1hUa3Nnbkc                
           iLCJpYXQiOjE2OTA5OTk1OTcsImV4cCI6MTY5MTA4NTk5N30.KVeeMyIUseEnlwONB7Vk1xV80AKzMl5Vu4lmwQQ                
           wkcB4cwNraRiD4P6NK_0S2A6648-p-7jtHMC_ifP3D7T2Mw',                                                       
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUW                
           lGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjAxN2E1MjQyLTUxOWUtNDFlOC05ODNkLWFhNTk5ZGU                
           wNDI5YyIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1                
           tb2NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo                
           5S2Jvd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0                
           yZXNhOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25                
           HIiwiYXVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J                
           vd28xTVlwQ0p3TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0yZXN                
           hOEZMd1ozNWk1bVJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25HIiw                
           iaWF0IjoxNjkwOTk5NTk3LCJleHAiOjE2OTA5OTk3MTd9.zUocxDQaGUuIw-kDM5cNkZv9BZnG1mGpgpDBgYh4ZM                
           DoB3VUNnf1wDmKnK42x7mSkUSsERwV2YR1GKjACKm-ag',                                                          
               c_nonce='04e5e6d0-3cd9-4abe-a19c-361062565dc1',                                                     
               c_nonce_expires_in=86400,  

## 1.3 Request credential (same device)

In [6]:
from eudi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
credential_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
print("Credential response: ", credential_response)

[23:36:40] Credential request JWT:                                                                   ]8;id=511737;file:///tmp/ipykernel_70125/621865142.py\621865142.py]8;;\:]8;id=895672;file:///tmp/ipykernel_70125/621865142.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVW               
           jNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUz               
           FGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0RRZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaEx               
           nWFRrc2duRyN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2Jvd28xTVlwQ0p3               
           TmZ6ZkZnZ1h1V3hkUEFkQVdoa3A5WHNrVTJjaXI3cjIxQXhxTjJUTXZNVFVTMUZtYU5XTG0yZXNhOEZMd1ozNWk1b               
           VJ6VXlnRFFkWlJTMWdDNDJBbTlEb3loNjhIcDFLc2FwOWU5eHplYkRZRmhoTGdYVGtzZ25HIiwidHlwIjoib3Blbm               
           lkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h               
           bmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjkxMDg2MDAwLCJpYXQiOjE2OTA5OTk2MDAsImlzcyI6ImRpZDprZ               
           Xk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2               
           dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUzFGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0R               
           RZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaExnWFRrc2duRyIsIm5vbmNlIjoiMDRlNWU2ZDAt               
           M2NkOS00YWJlLWExOWMtMzYxMDYyNTY1ZGMxIn0.qT7s5UqHt353KTHIhCRCZuQZsEf_686YHneNNjBieijI0Up7Z               
           6Zq_uHml_n-96RiYZZnRruEXU1_5LN0R3QaeA                                                                   

Credential response:  {'format': 'jwt_vc', 'credential': 'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzJiOWIwYTlhLTk0ZGUtNDk4MC1hYzgxLTQ0YTc1ODk5NTU4YSIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUtib3dvMU1ZcENKd05memZGZ2dYdVd4ZFBBZEFXaGtwOVhza1UyY2lyN3IyMUF4cU4yVE12TVRVUzFGbWFOV0xtMmVzYThGTHdaMzVpNW1SelV5Z0RRZFpSUzFnQzQyQW05RG95aDY4SHAxS3NhcDllOXh6ZWJEWUZoaExnWFRrc2duRyIsImlzcyI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjkwOTk5NjAwLCJleHAiOjE2OTEwODYwMDAsImlhdCI6MTY5MDk5OTYwMCwidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwiaWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzJiOWIwYTlhLTk0ZGUtNDk4MC1hYzgxLTQ0YTc1ODk5NTU4YSIsInR5cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldFNhbWVQcmVBdXRob3Jpc2VkIl0sImlzc3VlciI6ImRpZDplYnNpOnpoSkFSalBONjljRXR